In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [5]:
df = pd.read_csv(r'C:\Users\elifn\sentetik_veri.csv')
df.head()

,Document ID,PostAllergyWarning,PostDate,PostDescription,PostExpiry,PostFoodProvider,PostFoodType,PostPhotos,PostPrice,PostQuantity,PostTitle,PostId
0,IM0c8qrulbp6,Dairy-Free,2024-04-29 11:23:14.594594592,1 kg not rotten but it's about to go bad,2024-05-17 11:23:14.594594592,def789ghi123,Produce,images/c929dab2-73a0-4d75-a127-22dbeec3e1ed.jpeg,14,8,Gluten-free Pasta,IM0c8qrulbp6
1,ncXTZtj9TTPn,Dairy-Free,2023-10-10 19:01:37.297297296,2 kg not rotten but it's about to go bad,2023-10-28 19:01:37.297297296,22kjTPVeRbV4ErlXxNvKfW7ARmm1,Grains,images/c929dab2-73a0-4d75-a127-22dbeec3e1ed.jpeg,4,5,Organic Apple,ncXTZtj9TTPn
2,epMkn6OzcSZZ,Nut-Free,2023-06-16 14:33:30.810810810,4 kg not rotten but it's about to go bad,2023-07-04 14:33:30.810810810,22kjTPVeRbV4ErlXxNvKfW7ARmm1,Grains,images/c929dab2-73a0-4d75-a127-22dbeec3e1ed.jpeg,48,17,Gluten-free Pasta,epMkn6OzcSZZ
3,r3XJbq2ZoxR2,Gluten-Free,2023-06-12 05:20:00.000000000,8 kg not rotten but it's about to go bad,2023-06-30 05:20:00.000000000,22kjTPVeRbV4ErlXxNvKfW7ARmm1,Dairy,images/c929dab2-73a0-4d75-a127-22dbeec3e1ed.jpeg,77,11,Vegan Protein Bars,r3XJbq2ZoxR2
4,bgavuNEI12SL,Gluten-Free,2024-01-15 05:58:55.135135132,10 kg not rotten but it's about to go bad,2024-02-02 05:58:55.135135132,def789ghi123,Dairy,images/c929dab2-73a0-4d75-a127-22dbeec3e1ed.jpeg,18,15,Gluten-free Pasta,bgavuNEI12SL


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse


df = pd.read_csv(r'C:\Users\elifn\sentetik_veri.csv')


text_features = ['PostAllergyWarning', 'PostFoodProvider', 'PostDescription']  
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df[text_features].apply(lambda x: ' '.join(x), axis=1))


numeric_features = ['PostPrice', 'PostQuantity']
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(df[numeric_features])


merged_features = sparse.hstack((tfidf_matrix, scaled_numeric))


similarity_matrix = cosine_similarity(merged_features, merged_features)

# Öneri fonksiyonu
def get_recommendations(features, k=5):
    # Girdi özelliklerini işle
    text_data = ' '.join([str(features[feat]) for feat in text_features])
    tfidf_input = tfidf_vectorizer.transform([text_data])
    numeric_input = scaler.transform([[features[feat] for feat in numeric_features]])
    input_features = sparse.hstack((tfidf_input, numeric_input))
    
    # Benzerlik skorlarını hesapla
    similarity_scores = cosine_similarity(input_features, merged_features)
    
    # En yakın k ürünü seç
    similar_indices = similarity_scores.argsort()[0][-k-1:-1][::-1]
    recommended_products = df.iloc[similar_indices]['PostTitle']
    return recommended_products

# Örnek kullanım
user_features = {
    'PostAllergyWarning': 'Gluten-Free',
    'PostFoodProvider': 'Organic Market',
    'PostDescription': 'Freshly baked organic bread, free of gluten and preservatives',
    'PostPrice': 8.99,
    'PostQuantity': 20
}

recommendations = get_recommendations(user_features)
print(recommendations)


942    Vegan Protein Bars
684     Gluten-free Pasta
227       Fresh Salad Mix
518         Organic Apple
651     Gluten-free Pasta
Name: PostTitle, dtype: object


C:\Users\elifn\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
